<a href="https://colab.research.google.com/github/ozturkcemal/SupplyChainAnalytics/blob/main/04_SetCovering_Packing_Partitioning/02_SetPacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing library to read xlsx files
!pip install pandas openpyxl

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

# Read the data from the Excel file, specifying the correct header row
df = pd.read_excel('input.xlsx', header=1)  # This assumes the second row (index 1) is your header

# Drop the 'Sites' column, as requested
df = df.loc[:, df.columns != 'Sites']

# Print the cleaned dataframe
print(df.to_string())


# Read the data from the Excel file, specifying the correct header row
df2 = pd.read_excel('cost.xlsx')  # This assumes the second row (index 1) is your header

# Drop the 'District' column, as requested
df2 = df2.loc[:, df2.columns != 'District']

# Print the cleaned dataframe
print(df2.to_string())

In [ ]:
# Create a list 'sites' with numbers from 0 to the number of rows in the dataframe
sites = list(range(0, len(df)))
# Print the list 'sites' to verify
print("sites",sites)
# Create a list 'districts' with numbers from 0 to the number of columns in the dataframe
districts = list(range(0, len(df.columns)))
# Print the list 'districts' to verify
print("districts:",districts)
# Export the dataframe to a CSV file and open it in an editor
#df.to_csv('output.csv')

In [ ]:
import numpy as np
#preparing the cost array and populating
cost=np.zeros(len(districts),dtype=float)
for j in range(len(districts)):
  cost[j]=df2.iloc[j,0]
print("cost",cost)


# Create the 'cover' array with the same shape as the dataframe
cover = np.zeros((len(sites), len(districts)), dtype=int)

# Iterate through the dataframe to fill the 'cover' array
for i in range(len(sites)):
    for j in range(len(districts)):
        cover[i][j] = df.iloc[i, j]  # Get the value from the dataframe and assign it to the cover array

# Print the 'cover' array to verify
print("cover array:")
print(cover)

In [ ]:
#installing pyomo
!pip install -q pyomo

In [ ]:
#installing coin or
!apt-get install -y -qq coinor-cbc

In [ ]:
#importing pyomo environment
import pyomo.environ as pyo

In [ ]:
#creating a model object
model = pyo.ConcreteModel()

In [ ]:
#defining variables
model.x = pyo.Var(sites, within=pyo.Binary)
model.y = pyo.Var(districts, within=pyo.Binary)
model.x.pprint()
model.y.pprint()

In [ ]:
# Remove any existing objective component named "obj"
if hasattr(model, 'obj'):
    model.del_component('obj')

#defining objective function
model.obj = pyo.Objective(
    expr = sum(cost[j]*model.y[j] for j in districts),
    sense = pyo.minimize )
model.obj.pprint()

In [ ]:
#modeling the coverage constraints
model.constraint_coverage = pyo.ConstraintList()
for j in districts:
  model.constraint_coverage.add(sum(model.x[i] for i in sites if cover[i][j]>0)+model.y[j] >=1)
model.constraint_coverage.pprint()

In [ ]:
model.constraint_packing = pyo.ConstraintList()
model.constraint_packing.add(sum(model.x[i] for i in sites)<=4)
model.constraint_packing.pprint()

In [ ]:
#connecting cbc solver and printing the model
opt = pyo.SolverFactory('cbc')
model.pprint()

In [ ]:
#solve the model
opt_solution = opt.solve(model)

In [ ]:
# Print the values of the decision variables
#we are shifting the indices for presentation
print("\nEMS sites opening:")
for i in sites:
    if model.x[i].value > 0:
      print(f"{i+1}: {model.x[i].value}")

print("\nNot covered districts:")
for j in districts:
    if model.y[j].value > 0:
      print(f"{j+1}: {model.y[j].value}")

print(f"Total cost of uncovered districts: {model.obj()}")